In [1]:
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

import wandb
from wandb.integration.keras import WandbCallback
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K

path_to_snp = "../data/SNP/"

2024-12-02 15:00:11.868132: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
config={
        "epochs": 100,
        "batch_size": 32,
        "learning_rate": 1e-5,
        "architecture": "DeepSNP",
        "dropout": 0.2,
        # "l2": 1e-2,
        "seed": None,
}

In [3]:
wandb.init(
    project="SNP_alone_feature_selected_mi_DeepSNP",
    name=f"DeepSNP_pl_1_32_5_2",
    config=config
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jaredzwx221 (jaredzwx221-carnegie-mellon-univeristy). Use `wandb login --relogin` to force relogin


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

# def improved_model(input_shape, output_classes):
#     model = Sequential([
#         Dense(128, input_shape=(input_shape,), kernel_regularizer=l2(0.001)),
#         LeakyReLU(alpha=0.1),
#         Dropout(0.5),
#         Dense(64, kernel_regularizer=l2(1e-6)),
#         LeakyReLU(alpha=0.1),
#         Dropout(0.5),
#         Dense(32, kernel_regularizer=l2(1e-6)),
#         LeakyReLU(alpha=0.1),
#         Dropout(0.3),
#         Dense(32, kernel_regularizer=l2(1e-6)),
#         LeakyReLU(alpha=0.1),
#         Dropout(0.3),
#         Dense(output_classes, activation='softmax')
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=config['learning_rate'], clipvalue=1.0),
#         loss="sparse_categorical_crossentropy",
#         metrics=["sparse_categorical_accuracy"]
#     )
#     return model

def shallow_model(input_shape, output_classes):
    model = Sequential([
        Dense(128, input_shape=(input_shape,), activation="relu", kernel_regularizer=l2(config['l2'])),
        Dropout(config["dropout"]),
        # Dense(64, activation="relu", kernel_regularizer=l2(config['l2'])),
        Dense(64, activation="relu"),
        Dropout(config["dropout"]),
        Dense(32, activation="relu"),
        Dropout(config["dropout"]-0.1),
        Dense(32, activation="relu"),
        Dropout(config["dropout"]-0.1),
        Dense(output_classes, activation="softmax")
    ])
    model.compile(
        optimizer=Adam(learning_rate=config['learning_rate']),
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

def even_shallow_model(input_shape, output_classes):
    model = Sequential([
        Dense(64, input_shape=(input_shape,), activation="relu"),
        Dropout(config["dropout"]),
        # Dense(64, activation="relu", kernel_regularizer=l2(config['l2'])),
        Dense(32, activation="relu"),
        Dropout(config["dropout"]),
        Dense(16, activation="relu"),
        Dropout(config["dropout"]-0.1),
        Dense(16, activation="relu"),
        Dropout(config["dropout"]-0.1),
        Dense(output_classes, activation="softmax")
    ])
    model.compile(
        optimizer=Adam(learning_rate=config['learning_rate']),
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

def DeepSNP(input_shape, output_classes):
    model = Sequential([
        Conv1D(64, kernel_size=5, activation="relu", input_shape=(input_shape, 1)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(config["dropout"]),

        Conv1D(128, kernel_size=3, activation="relu"),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(config["dropout"]),

        Flatten(),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(config["dropout"]),

        Dense(output_classes, activation="softmax")
    ])

    model.compile(
        optimizer=Adam(learning_rate=config['learning_rate']),
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

def evaluate_model(model, X_test, y_test):
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

    predictions = model.predict(X_test)
    true_labels = np.argmax(to_categorical(y_test, num_classes=3), axis=1)
    predicted_labels = np.argmax(predictions, axis=1)

    report = classification_report(true_labels, predicted_labels, output_dict=True)
    return score[1], report["macro avg"]["precision"], report["macro avg"]["recall"], report["macro avg"]["f1-score"]

In [5]:
vcf_sele = pd.read_pickle(path_to_snp + "vcf_select_mi.pkl")

vcf_sele

,label,Subject,Group,12373_89763,56273,367261_81730,105448,375979,521667,100275_124934,...,211299,355098_174869,9425_152424,138682_109508,261887,88681,249988,11232,78936,415684
0,0,011_S_0002,0.0,0,0,0,0,0,1,0,...,0,0,0,0,2,0,0,2,0,0
3,1,011_S_0002,0.0,0,0,0,0,0,1,0,...,0,0,0,0,2,0,0,2,0,0
5,0,011_S_0008,0.0,0,0,0,0,0,2,0,...,0,0,0,0,1,1,0,2,0,0
8,1,011_S_0008,0.0,0,0,0,0,0,2,0,...,0,0,0,0,1,1,0,2,0,0
9,0,100_S_0015,0.0,0,0,2,0,0,1,0,...,0,0,0,0,2,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,1,072_S_4610,1.0,0,1,0,0,0,2,0,...,0,0,0,0,1,1,0,2,0,0
1699,1,072_S_4613,1.0,0,0,0,0,0,1,0,...,0,0,0,0,2,2,0,0,0,1
1700,1,153_S_4621,1.0,0,2,1,1,0,0,0,...,0,1,0,0,2,0,0,1,0,2
1702,2,005_S_4707,2.0,0,1,1,0,0,1,0,...,1,0,0,0,2,1,1,2,0,1


In [6]:
cols = list(set(vcf_sele.columns) - set(["Subject", "Group", "label"]))
X = vcf_sele[cols]
y = vcf_sele["label"].astype(int)

X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y)
y_df.rename(columns={"label": "label"}, inplace=True)

data = pd.concat([X_df, y_df], axis=1)

# Over-sampling
class_counts = y_df["label"].value_counts()
max_count = class_counts.max()

oversampled_data = []
for label, count in class_counts.items():
    class_data = data[data["label"] == label]
    if count < max_count:
        sampled_data = class_data.sample(max_count - count, replace=True, random_state=42)
        oversampled_data.append(sampled_data)

oversampled_data = pd.concat(oversampled_data, axis=0)
balanced_data = pd.concat([data, oversampled_data], axis=0)

X_balanced = balanced_data.iloc[:, :-1]
y_balanced = balanced_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.1, stratify=y_balanced, random_state=42
)

print("After sampling and splitting:")
print("Train label distribution:")
print(y_train.value_counts() / len(y_train))
print("Test label distribution:")
print(y_test.value_counts() / len(y_test))

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
assert list(X_train.columns) == list(X_test.columns), "Column names do not match between X_train and X_test"

After sampling and splitting:
Train label distribution:
label
0    0.333929
1    0.333036
2    0.333036
Name: count, dtype: float64
Test label distribution:
label
2    0.336
1    0.336
0    0.328
Name: count, dtype: float64
(1120, 16333)
(1120,)
(125, 16333)
(125,)


In [7]:
# cols = list(set(vcf_sele.columns) - set(["Subject", "Group", "label"]))
# X = vcf_sele[cols]
# y = vcf_sele["label"].astype(int).values

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

# y_train = pd.DataFrame(y_train)
# y_test = pd.DataFrame(y_test)

In [9]:
# X_train = X_train.reset_index(drop=True)
# y_train = y_train.reset_index(drop=True)

# print("Before sampling:")
# print(y_train.value_counts()/len(y_train))
# X_train_df = pd.DataFrame(X_train)
# y_train_df = pd.DataFrame(y_train)
# y_train_df.rename(columns={0: 'label'}, inplace=True)
# original_columns = X_train_df.columns
# data = pd.concat([X_train_df, y_train_df], axis=1)

# class_counts = y_train_df["label"].value_counts()
# max_count = class_counts.max()

# oversampled_data = []
# for label, count in class_counts.items():
#     class_data = data[data["label"] == label]
    
#     if count < max_count:
#         sampled_data = class_data.sample(max_count - count, replace=True, random_state=42)
#         oversampled_data.append(sampled_data)

# oversampled_data = pd.concat(oversampled_data, axis=0)
# balanced_data = pd.concat([data, oversampled_data], axis=0)

# X_train = balanced_data.iloc[:, :-1].values
# y_train = balanced_data.iloc[:, -1].values

# X_train = pd.DataFrame(X_train)
# y_train = pd.DataFrame(y_train)

# X_train.columns = original_columns
# y_train.rename(columns={0: 'label'}, inplace=True)

# print("After sampling:")
# print(y_train.value_counts()/len(y_train))

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
# assert list(X_train.columns) == list(X_test.columns), "Column names do not match between X_train and X_test"

In [10]:
# X_train = pd.read_pickle(path_to_snp + "X_train_vcf.pkl")
# y_train = pd.read_pickle(path_to_snp + "y_train_vcf.pkl")
# X_test = pd.read_pickle(path_to_snp + "X_test_vcf.pkl")
# y_test = pd.read_pickle(path_to_snp + "y_test_vcf.pkl")

In [11]:
X_train.columns, X_test.columns

(Index(['190641', '166764', '507965', '162937', '175951', '216154',
        '156847_139816', '10043', '27574_67442', '421463',
        ...
        '69073', '585184', '96051', '514217', '241315', '131733',
        '147775_73365', '508050', '279543', '187339'],
       dtype='object', length=16333),
 Index(['190641', '166764', '507965', '162937', '175951', '216154',
        '156847_139816', '10043', '27574_67442', '421463',
        ...
        '69073', '585184', '96051', '514217', '241315', '131733',
        '147775_73365', '508050', '279543', '187339'],
       dtype='object', length=16333))

In [12]:
print(y_test.value_counts()/len(y_test))

label
2    0.336
1    0.336
0    0.328
Name: count, dtype: float64


In [13]:
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weight_dict = dict(enumerate(class_weights))
# class_weight_dict

In [14]:
acc, precision, recall, f1 = [], [], [], []

In [16]:
seeds = [87, 49, 114, 38, 4]

In [17]:
# seeds = random.sample(range(1, 200), 5)

# seeds

In [18]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# def lr_schedule(epoch, lr):
#     initial_lr = 0.001
#     decay_rate = 0.1
#     decay_epoch = 10
#     if epoch % decay_epoch == 0 and epoch != 0:
#         return lr * decay_rate
#     return lr

# lr_scheduler = LearningRateScheduler(lr_schedule)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

class LearningRateLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        optimizer = self.model.optimizer
        current_lr = float(K.get_value(optimizer.learning_rate))
        
        print(f"\nEpoch {epoch + 1}: Current Learning Rate is {current_lr:.6f}")
        
        wandb.log({"learning_rate": current_lr})

In [19]:
# Train
for seed in seeds:
    reset_random_seeds(seed)
    print(f"Training model with seed {seed}...")
    wandb.config.seed = seed

    # Build and train model
    model = DeepSNP(input_shape=X_train.shape[1], output_classes=3)
    history = model.fit(
        X_train, y_train,
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        validation_split=0.1,
        # class_weight=class_weight_dict,
        verbose=1,
        callbacks=[WandbCallback(
            monitor='val_loss',
            save_model=False,
            log_weights=False,
            log_gradients=False
        ), reduce_lr, LearningRateLogger()]
    )
    
    # Evaluate model
    accuracy, prec, rec, f1_score = evaluate_model(model, X_test, y_test)
    acc.append(accuracy)
    precision.append(prec)
    recall.append(rec)
    f1.append(f1_score)
    
    # Log metrics to wandb
    wandb.log({
        "accuracy": accuracy,
        "precision": prec,
        "recall": rec,
        "f1_score": f1_score,
    })
    
wandb.finish()

Training model with seed 87...


2024-12-02 15:00:26.399484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 15:00:27.109861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20825 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:dd:00.0, compute capability: 7.5
wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


Epoch 1/100


2024-12-02 15:00:29.632065: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


32/32 [==============================] - ETA: 0s - loss: 1.5819 - sparse_categorical_accuracy: 0.3631
Epoch 1: Current Learning Rate is 0.000010
32/32 [==============================] - 5s 67ms/step - loss: 1.5819 - sparse_categorical_accuracy: 0.3631 - val_loss: 1.1358 - val_sparse_categorical_accuracy: 0.3571 - lr: 1.0000e-05
Epoch 2/100
31/32 [============================>.] - ETA: 0s - loss: 0.7247 - sparse_categorical_accuracy: 0.6986
Epoch 2: Current Learning Rate is 0.000010
32/32 [==============================] - 2s 57ms/step - loss: 0.7267 - sparse_categorical_accuracy: 0.6994 - val_loss: 1.6891 - val_sparse_categorical_accuracy: 0.3571 - lr: 1.0000e-05
Epoch 3/100
31/32 [============================>.] - ETA: 0s - loss: 0.5538 - sparse_categorical_accuracy: 0.7833
Epoch 3: Current Learning Rate is 0.000010
32/32 [==============================] - 2s 56ms/step - loss: 0.5557 - sparse_categorical_accuracy: 0.7817 - val_loss: 2.3692 - val_sparse_categorical_accuracy: 0.3571 - l

accuracy,▃█▅▁▄
epoch,▂▄▅▆▆▇█▁▃▃▅▅▆▇▂▄▅▅▅▆▁▂▂▃▃▄▅▆▆▇▂▃▄▄▄▅▆▆▆█
f1_score,▂█▅▁▄
learning_rate,█▄▂▁▁▁▁▁▁▁█▄▃▁▁▁▁▁▁▁█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▂▁▂▁▇▂▁▂▂▁▂▁▂█▂▁▁▁▁▁▁▂▅▃▂▁▂▁▂▁▁▁▄▂▂▂▁▁▁▂
precision,▁█▆▂▄
recall,▃█▅▁▄
sparse_categorical_accuracy,▇▇████▁▇████████████████▇███████████████
val_loss,▁▁▁▁▁▁▁▁▁▁▂█▂▂▂▂▂▂▂▁▂▂▂▂▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_sparse_categorical_accuracy,█▇▇▇▇▃▃▅▆▇▇▇▇▇▇▇▃▁▇▇▇██████▇▇▇▇▃▆▇█▇▇▇▇▇
accuracy,0.512


In [20]:
print("\nPerformance Metrics Across Seeds:")
print(f"Avg accuracy: {np.mean(acc):.4f} | Std: {np.std(acc):.4f}")
print(f"Avg precision: {np.mean(precision):.4f} | Std: {np.std(precision):.4f}")
print(f"Avg recall: {np.mean(recall):.4f} | Std: {np.std(recall):.4f}")
print(f"Avg F1-score: {np.mean(f1):.4f} | Std: {np.std(f1):.4f}")


Performance Metrics Across Seeds:
Avg accuracy: 0.5200 | Std: 0.0299
Avg precision: 0.5006 | Std: 0.0363
Avg recall: 0.5221 | Std: 0.0294
Avg F1-score: 0.4967 | Std: 0.0389


In [ ]:
seeds

In [ ]:
# Plot metrics for each seed
plt.figure(figsize=(10, 6))
plt.plot(range(len(seeds)), acc, marker='o', label='Accuracy')
plt.plot(range(len(seeds)), precision, marker='o', label='Precision')
plt.plot(range(len(seeds)), recall, marker='o', label='Recall')
plt.plot(range(len(seeds)), f1, marker='o', label='F1-score')
plt.title("Performance Metrics Across Random Seeds")
plt.xlabel("Random Seed")
plt.ylabel("Score")
plt.legend()
plt.grid()
plt.show()